# Stop Sign Counting By Zipcodes in California

This notebook demonstrates the use of geodataframe joins with point in polygon in cuSpatial

## Prerequisite: Datasets

Dataset used (TODO: discuss if need to self host):
1. Stops (Signs and Stop lines) Dataset from OpenStreetMap: [Tag Page](https://wiki.openstreetmap.org/wiki/Tag:highway%3Dstop)
2. USA ZipCode: [Download](https://catalog.data.gov/dataset/tiger-line-shapefile-2019-2010-nation-u-s-2010-census-5-digit-zip-code-tabulation-area-zcta5-na)
3. USA States boundary: [link](https://osm-boundaries.com/)

Download the datasets and save as:
1. USA_Stops_Vertices.csv
2. USA_Zipcodes_2019_Tiger.csv
3. USA_States.csv

## Overview

Stop signs is a important symbol of city and road development in geographical information systems. This notebook processes all stop sign locations from the a dataset, using spatial joins to categorify them within the zipcodes boundaries that are located within California. This notebook does the following steps:
1. Filters the zipcode boundaries that locates in California with spatial join.
2. Filters the stop signs that locates in all the zipcodes with spatial join.
3. Count the stop signs by zipcodes.
4. Visualize the result on map.

In [1]:
# Import Necessary Packages
import cuspatial, cudf
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from tqdm.notebook import trange
import pydeck as pdk

## Load Dataset and Cleanup

We load the dataset and store them as device dataframe in cuspatial. Note that the second cell below loads the dataset with cuDF, then adopts geopandas to parse the WKT strings into shapely objects. This is a slow step performed on CPU and requires data transfer between device and host.

In [2]:
# Import Stop Sign CSV
d_stops = cudf.read_csv("USA_Stops_Vertices.csv", usecols=["x", "y"])

In [3]:
#Import CSV of Zipcodes
d_zip = cudf.read_csv(
    "USA_Zipcodes_2019_Tiger.csv",
    usecols=["WKT", "ZCTA5CE10", "INTPTLAT10", "INTPTLON10"])
d_zip.INTPTLAT10 = d_zip.INTPTLAT10.astype("float")
d_zip.INTPTLON10 = d_zip.INTPTLON10.astype("float")

In [4]:
# Load WKT as shapely objects
h_zip = d_zip.to_pandas()
h_zip["WKT"] = h_zip["WKT"].apply(wkt.loads)
h_zip = gpd.GeoDataFrame(h_zip, geometry="WKT", crs='epsg:4326')

# Roundtrip back to geopandas
d_zip = cuspatial.from_geopandas(h_zip)

In [5]:
# Load State Boundaries
states = gpd.read_file("USA_States.csv", geometry='WKT', crs='epsg:4326')
d_states = cuspatial.from_geopandas(states)

In [6]:
class QuadTree:
    """Helper class to use cuspatial quadtree interface
    """
    def __init__(self,
                 df,
                 x_column,
                 y_column,
                 x_min=None,
                 x_max=None,
                 y_min=None,
                 y_max=None,
                 scale = -1,
                 max_depth = 15,
                 min_size = 12):

        self.x_min = df[x_column].min() if not x_min else x_min
        self.x_max = df[x_column].max() if not x_max else x_max
        self.y_min = df[y_column].min() if not y_min else y_min
        self.y_max = df[y_column].max() if not y_max else y_max
        
        self.scale = scale
        self.max_depth = max_depth
        self.min_size = min_size

        self.point_df = df
        self.x_column = x_column
        self.y_column = y_column
        
        self.polygon_point_mapping = None
        
        self.point_indices, self.quadtree = (
            cuspatial.quadtree_on_points(df[x_column],
                                         df[y_column],
                                         self.x_min,
                                         self.x_max,
                                         self.y_min,
                                         self.y_max,
                                         self.scale,
                                         self.max_depth,
                                         self.min_size))

    def set_polygon(self, df, poly_column):
        polys = df[poly_column]

        parts = polys.polygons.part_offset
        rings = polys.polygons.ring_offset
        x = polys.polygons.x
        y = polys.polygons.y

        geometries = cudf.Series(polys.polygons.geometry_offset)
            
        poly_bboxes = cuspatial.polygon_bounding_boxes(
            parts,
            rings,
            x,
            y
        )
        intersections = cuspatial.join_quadtree_and_bounding_boxes(
            self.quadtree, poly_bboxes, self.x_min, self.x_max, self.y_min, self.y_max, self.scale, self.max_depth
        )
        polygon_point_mapping = cuspatial.quadtree_point_in_polygon(
            intersections,
            self.quadtree,
            self.point_indices,
            self.point_df[self.x_column],
            self.point_df[self.y_column],
            parts,
            rings,
            x,
            y
        )

        # Update Polygon Index to MultiPolygon Index
        polygon_index = geometries.searchsorted(polygon_point_mapping.polygon_index, side="right")-1
        polygon_point_mapping.polygon_index = polygon_index

        self.polygon_point_mapping = polygon_point_mapping.reset_index(drop=True)
        
        # Remap point indices
        idx_of_idx = self.point_indices.take(
            self.polygon_point_mapping.point_index
        ).reset_index(drop=True)
        self.polygon_point_mapping.point_index = idx_of_idx

        self.polygon_df = df

    def _subset_geodf(self, geodf, columns):
        res = cudf.DataFrame()
        for col in columns:
            res[col] = geodf[col]
        return res

    def points(self, columns = None):
        if self.polygon_point_mapping is None:
            raise ValueError("First set polygon dataframe.")
        
        if not columns:
            df = self.point_df
        else:
            df = self._subset_geodf(self.point_df, columns)

        if any(dtype == "geometry" for dtype in df.dtypes):
            df = cuspatial.GeoDataFrame(df)
        
        mapping = self.polygon_point_mapping
        res = df.iloc[mapping.point_index]
        res = res.reset_index(drop=True)
        res["polygon_index"] = mapping.polygon_index
        res["point_index"] = mapping.point_index
        return res

    def polygons(self, columns = None):
        if self.polygon_point_mapping is None:
            raise ValueError("First set polygon dataframe.")
        
        if not columns:
            df = self.polygon_df
        else:
            df = self._subset_geodf(self.polygon_df, columns)
        
        if any(dtype == "geometry" for dtype in df.dtypes):
            df = cuspatial.GeoDataFrame(df)
        
        mapping = self.polygon_point_mapping
        res = df.iloc[mapping.polygon_index]
        res = res.reset_index(drop=True)
        res["polygon_index"] = mapping.polygon_index
        res["point_index"] = mapping.point_index
        return res
    
    def point_left_join_polygon(self, point_columns=None, polygon_columns=None):
        points = self.points(point_columns)
        polygons = self.polygons(polygon_columns)
        joined = points.merge(polygons, on=["polygon_index", "point_index"], how="left")
        joined = joined.drop(["polygon_index", "point_index"], axis=1)
        return cuspatial.GeoDataFrame(joined)

## Filtering Zipcode by its Geometric Center

The Zipcode Dataset contains boundaries for all zipcodes in the US. The below uses the geometric center for each zipcode and uses cuspatial's quadtree interface to filter zipcodes that locates only in California.

In [7]:
# Use quadtree to filter zip codes

# Build a point quadtree using the geometric center of the zip code region
zipcode_quadtree = QuadTree(d_zip, x_column="INTPTLON10", y_column="INTPTLAT10")

# Pass boundary
zipcode_quadtree.set_polygon(d_states, poly_column='geometry')

# Join state and zip code boundaries
zipcode_by_state = zipcode_quadtree.point_left_join_polygon(["WKT", "ZCTA5CE10"], ["STUSPS"])

# Get Californian zipcodes
CA_zipcode = zipcode_by_state[zipcode_by_state.STUSPS == 'CA']

/raid/wangm/dev/rapids/cuspatial/python/cuspatial/cuspatial/core/spatial/indexing.py:174: UserWarning: scale -1 is less than required minimum scale 0.009837776664632286. Clamping to minimum scale
  warnings.warn(
/raid/wangm/dev/rapids/cuspatial/python/cuspatial/cuspatial/core/spatial/join.py:171: UserWarning: scale -1 is less than required minimum scale 0.009837776664632286. Clamping to minimum scale
  warnings.warn(


In [8]:
len(CA_zipcode), len(d_stops)

(1762, 467666)

In [9]:
len(d_zip)

33144

## Join stop signs dataset with California Zipcode boundaries

The below joins the stop sign dataset (460K data points) with all zip code boundaries in California (1700 data points).

In [10]:
# Build a 2nd quadtree with all stop signs in US
stop_quadtree = QuadTree(d_stops, x_column='x', y_column='y')

# Pass zip code polygons
stop_quadtree.set_polygon(CA_zipcode, poly_column="WKT")

# Join the stop signs and the zip code dataframe
stop_by_zipcode = stop_quadtree.point_left_join_polygon(["x", "y"], ["ZCTA5CE10"])

/raid/wangm/dev/rapids/cuspatial/python/cuspatial/cuspatial/core/spatial/indexing.py:174: UserWarning: scale -1 is less than required minimum scale 0.0029100948550503493. Clamping to minimum scale
  warnings.warn(
/raid/wangm/dev/rapids/cuspatial/python/cuspatial/cuspatial/core/spatial/join.py:171: UserWarning: scale -1 is less than required minimum scale 0.0029100948550503493. Clamping to minimum scale
  warnings.warn(


In [11]:
stop_by_zipcode.head()

,x,y,ZCTA5CE10
0,-117.842624,33.639451,92617
1,-117.842764,33.639535,92617
2,-117.843027,33.639094,92617
3,-117.842878,33.639017,92617
4,-117.838910,33.635505,92617


## Zipcode counting with cuDF

The below resorts to [cuDF](https://docs.rapids.ai/api/cudf/stable/index.html) and counts the number of stop signs per zip code. Then merge the geometry information from the zipcode dataset.

In [12]:
# Count the Stop Signs by California Zip Codes
stop_counts = stop_by_zipcode.groupby("ZCTA5CE10").x.count().rename("stop_count")
stop_counts.head(), len(stop_counts)

(ZCTA5CE10
 94901    131
 94535    205
 95112    103
 95407    126
 93933    205
 Name: stop_count, dtype: int32,
 1352)

In [13]:
# Fetch the polygon boundaries
stop_counts_and_bounds = cuspatial.GeoDataFrame(CA_zipcode.merge(stop_counts, on="ZCTA5CE10", how="left"))
stop_counts_and_bounds["stop_count"] = stop_counts_and_bounds["stop_count"].astype("int").fillna(0)
print("DataFrame Size: ", len(stop_counts_and_bounds))

DataFrame Size:  1762


In [14]:
# Move dataframe to host for visualization
host_df = stop_counts_and_bounds.to_geopandas()
host_df = host_df.rename({"WKT": "geometry"}, axis=1)
host_df.head()

,geometry,ZCTA5CE10,STUSPS,stop_count
10758,"POLYGON ((-123.63668 39.15707, -123.63663 39.1...",95466,CA,4
10759,"POLYGON ((-123.80367 39.26962, -123.79691 39.2...",95410,CA,1
10763,"POLYGON ((-123.53414 39.29423, -123.53396 39.2...",95482,CA,98
10765,"POLYGON ((-123.27266 38.79839, -123.27187 38.7...",95425,CA,3
10768,"POLYGON ((-121.33482 38.70341, -121.33470 38.7...",95621,CA,96


## Visualization

Now, we visualize the stop sign count results using [PyDeck](https://deckgl.readthedocs.io/en/latest/index.html).

In [15]:
# Visualize the Dataset

# Geo Center of CA: 120°4.9'W 36°57.9'N
view_state = pdk.ViewState(
    **{"latitude": 33.96500, "longitude": -118.08167, "zoom": 6, "maxZoom": 16, "pitch": 95, "bearing": 0}
)

gpd_layer = pdk.Layer(
        "GeoJsonLayer",
        data=host_df[["geometry", "stop_count", "ZCTA5CE10"]],
        get_polygon="geometry",
        get_elevation="stop_count",
        extruded=True,
        elevation_scale=50,
        get_fill_color=[227,74,51],
        get_line_color=[255, 255, 255],
        auto_highlight=True,
        filled=True,
        wireframe=True,
        pickable=True
    )

tooltip = {"html": "<b>Stop Sign Count:</b> {stop_count} <br> <b>ZipCode: {ZCTA5CE10}"}

r = pdk.Deck(
    gpd_layer,
    initial_view_state=view_state,
    map_style=pdk.map_styles.LIGHT,
    tooltip=tooltip,
)

# Time consuming since it contains ~1700 polygons
r.to_html("geopandas_layer.html", notebook_display=False)

### Open geopandas_layer.html to see visulization result

![stop_per_state_map](stop_states.png)